In [ ]:
import numpy as np
import xgboost as xgb
import os
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# Use this to generate concatenated data file and label encoder from Video Dataset folder
# Assuming that you already used Data Extraction script on the dataset Folder.
# Note that every data file should be named after it's action name.

def genConcatData(directory="data/"):
    encoder = {}
    data = []
    i = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            path = os.path.join(root, file)
            if(file.endswith(".txt")):
                encoder[i] = file.replace(".txt","")            
                data.append(pd.read_csv(path,header=None).drop(0,axis=1).drop_duplicates())
                i += 1

    for i in range(len(encoder)):
        data[i][290] = i   
    pd.concat(data).to_csv("data.txt",header=None)
    pickle.dump(labels, open("labels.dat", "wb"),protocol=2)    
    

In [ ]:
# First column removed, because contains constant value
df = pd.read_csv("data.txt",header=None).drop(0,axis=1)

In [ ]:
df.shape

In [ ]:
# Initial Dataset was containing 290 features consisting of 14 body parts, 
# however we decided to remove 2 body parts which are Left and Right ankle
# This function generates indexes which should be removed from dataset 
# depending on input body part indexes.
# Here defaults are 10 and 13, which means that Left and Right ankle.
def genIdxsToRemove(indxs = [10,13]):      
    
    indxs = np.array(indxs)
    num_ids = indxs.shape[0]
    dist = 14*2+1
    idxsToRemove = []
    
    for idx in indxs:    
        for i in range(10):
            idxsToRemove.append(2*idx+i*dist+1)
            idxsToRemove.append(2*idx+i*dist+2)
            
    return idxsToRemove

In [ ]:
X = df.drop(genIdxsToRemove()+[290],axis=1).values
y = df[290].values

In [ ]:
X.shape

In [ ]:
### Code for cross-valiation 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

lamdas = np.logspace(1,5,5)
score_best = -10e15
for lamda in lamdas:
        print(lamda)
        clf = xgb.XGBRegressor(n_estimators=1000, learning_rate=1, gamma=0, n_jobs=4, reg_lambda=lamda,
                           colsample_bytree=1, max_depth=7)   
        clf.fit(X_train,y_train)
        test_score = accuracy_score(y_test,clf.predict(X_test))
        if(test_score>score_best):
            print(test_score)
            score_best = test_score
            best_lamda = lamda


In [ ]:
clf = xgb.XGBClassifier(n_estimators=1000, learning_rate=0.1,reg_lambda=best_lamda, n_jobs=4,max_depth=7) 
clf.fit(X,y)

In [ ]:
y_pred = clf.predict(X)

In [ ]:
accuracy_score(y,y_pred)

In [ ]:
# Saving Classifier in file.
pickle.dump(clf, open("clf.dat", "wb"),protocol=2)